In [13]:
import yfinance as yf
import talib
from stockstats import StockDataFrame as Sdf
import pickle
import os
import sys
import pandas as pd
import time
from datetime import datetime

In [2]:
from Historic_Crypto import HistoricalData
from Historic_Crypto import Cryptocurrencies
from Historic_Crypto import LiveCryptoData

In [16]:
def datetime_str_to_timestamp(datetime_str):
    try:
        dt = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
        return dt.strftime('%Y-%m-%d-%H-%M')
    except ValueError:
        print("Incorrect format for datetime string. Please provide it in 'YYYY-MM-DD HH:MM:SS' format.")


In [20]:
INDEX = ["BTC-USD", "ETH-USD", "LTC-USD", "DOGE-USD", "SOL-USD"]
#start_date = "2024-04-01-00-00"
#end_date = "2024-04-05-00-00"
start_date = datetime_str_to_timestamp("2022-04-30 00:00:00")
end_date   = datetime_str_to_timestamp("2022-05-01 00:00:00")

ticker = 'MATICUSDT'
# '5m' : 300
Temp = HistoricalData(ticker[:-4] + "-USD", 300, start_date=start_date, end_date=end_date, verbose=False).retrieve_data()

In [32]:
print(type(Temp))

<class 'pandas.core.frame.DataFrame'>


In [25]:
def datetime_to_timestamp(index):
    timestamps = []
    for dt in index:
        timestamp = int(time.mktime(dt.timetuple()) * 1000)
        timestamps.append(timestamp)
    return timestamps

In [22]:
print(type(Temp.index))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [28]:
Temp['timestamp'] = datetime_to_timestamp(Temp.index)
Temp['time'] = Temp.index

In [30]:
Temp.index = [x for x in range(len(Temp))]

In [31]:
Temp

,low,high,open,close,volume,time,timestamp
0,1.1350,1.1427,1.1407,1.1409,96061.0,2022-04-30 00:00:00,1651294800000
1,1.1400,1.1440,1.1411,1.1435,50014.2,2022-04-30 00:05:00,1651295100000
2,1.1343,1.1440,1.1438,1.1371,933008.3,2022-04-30 00:10:00,1651295400000
3,1.1363,1.1426,1.1370,1.1425,65555.3,2022-04-30 00:15:00,1651295700000
4,1.1418,1.1468,1.1426,1.1445,147751.2,2022-04-30 00:20:00,1651296000000
...,...,...,...,...,...,...,...
284,1.0292,1.0363,1.0331,1.0319,241667.5,2022-04-30 23:40:00,1651380000000
285,1.0300,1.0357,1.0316,1.0330,345636.1,2022-04-30 23:45:00,1651380300000
286,1.0276,1.0435,1.0332,1.0352,339401.7,2022-04-30 23:50:00,1651380600000
287,1.0317,1.0361,1.0349,1.0327,218047.3,2022-04-30 23:55:00,1651380900000


In [9]:
Temp2.index

DatetimeIndex(['2024-04-01 00:05:00', '2024-04-01 00:10:00',
               '2024-04-01 00:15:00', '2024-04-01 00:20:00',
               '2024-04-01 00:25:00', '2024-04-01 00:30:00',
               '2024-04-01 00:35:00', '2024-04-01 00:40:00',
               '2024-04-01 00:45:00', '2024-04-01 00:50:00',
               ...
               '2024-04-04 23:15:00', '2024-04-04 23:20:00',
               '2024-04-04 23:25:00', '2024-04-04 23:30:00',
               '2024-04-04 23:35:00', '2024-04-04 23:40:00',
               '2024-04-04 23:45:00', '2024-04-04 23:50:00',
               '2024-04-04 23:55:00', '2024-04-05 00:00:00'],
              dtype='datetime64[ns]', name='time', length=1152, freq=None)

In [ ]:
API_key = "X
private_key = 'X'

In [ ]:
import time
import requests
import hashlib
import hmac
import base64

In [33]:
import requests
import time
from datetime import datetime

def get_unix_time(date_string):
    """Converts date string to UNIX timestamp"""
    return int(time.mktime(datetime.strptime(date_string, '%Y-%m-%d').timetuple()))

def fetch_ohlc_data(pair, interval, start, end):
    """Fetches OHLC data from Kraken for given date range and interval"""
    url = "https://api.kraken.com/0/public/OHLC"
    data = []
    current_start = start

    while current_start < end:
        params = {
            'pair': pair,
            'interval': interval,
            'since': current_start
        }
        response = requests.get(url, params=params)
        result = response.json()
        
        if 'result' in result:
            for item in result['result'][pair]:
                data.append(item)
            last = result['result']['last']
            current_start = last
        else:
            break

        time.sleep(1)  # Respect API rate limits

    return data

# Usage example
pair = "XXBTZUSD"
interval = 5  # 5 minutes
start_date = "2023-01-01"
end_date = "2023-01-02"
start_unix = get_unix_time(start_date)
end_unix = get_unix_time(end_date)

ohlc_data = fetch_ohlc_data(pair, interval, start_unix, end_unix)
print(ohlc_data)

[]


In [41]:
import krakenex
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)
ohlc, last = k.get_ohlc_data("BCHUSD", interval=5, since=1603013580)
print(ohlc)

                           time    open    high     low   close    vwap  \
dtime                                                                     
2024-04-25 02:55:00  1714013700  478.28  478.65  478.28  478.65  478.47   
2024-04-25 02:50:00  1714013400  477.83  477.94  477.83  477.94  477.87   
2024-04-25 02:45:00  1714013100  478.95  478.95  477.68  477.68  477.92   
2024-04-25 02:40:00  1714012800  478.89  478.89  478.89  478.89  478.89   
2024-04-25 02:35:00  1714012500  478.79  478.81  478.79  478.81  478.79   
...                         ...     ...     ...     ...     ...     ...   
2024-04-22 15:20:00  1713799200  505.20  505.20  505.20  505.20    0.00   
2024-04-22 15:15:00  1713798900  505.13  505.20  505.13  505.20  505.16   
2024-04-22 15:10:00  1713798600  505.69  506.14  505.69  506.04  505.85   
2024-04-22 15:05:00  1713798300  505.45  505.45  505.04  505.25  505.32   
2024-04-22 15:00:00  1713798000  506.72  506.72  505.69  505.69  505.74   

                       v

In [37]:
help(KrakenAPI)

Help on class KrakenAPI in module pykrakenapi.pykrakenapi:

class KrakenAPI(builtins.object)
 |  KrakenAPI(api, tier='Intermediate', retry=1, crl_sleep=5)
 |  
 |  A python implementation of the Kraken API.
 |  
 |  Implements the Kraken API methods using the low-level krakenex python
 |  package. See
 |  https://www.kraken.com/help/api
 |  and
 |  https://github.com/veox/python3-krakenex
 |  
 |  Parameters
 |  ----------
 |  api : krakenex.API
 |      An instance of the krakenex.API class. A reference to the input
 |      is created and accessible via ``KrakenAPI.api``.
 |  
 |  tier : str, optional (default='Intermediate')
 |      Your Kraken tier level, used to adjust the limit of the call rate to
 |      the Kraken API in order to prevent 15 minute temporary lockouts.
 |      Must be one of {'None', 'Starter', 'Intermediate', 'Pro'}.
 |      Set tier='None' to disable the call rate limiter.
 |      See https://support.kraken.com/hc/en-us/articles/206548367.
 |  
 |  retry : float,